#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
!pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=18803 sha256=5b7e505bb3d885c812c017c4962a00a6f68ba7b62559e20b1497d23f46f1f289
  Stored in directory: /tmp/pip-ephem-wheel-cache-nsalehyp/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [3]:
import os
import pandas as pd

In [4]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


2023-09-25 14:17:54.849157: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 14:17:55.007695: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 14:17:55.008713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 14:17:55.711694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Get latest file URLS and Current Date Ranges

In [5]:
latest_URLS = Boston311LogReg.get311URLs()

In [6]:
print(latest_URLS)

{'2023': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmp1m2x5llz.csv', '2022': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv', '2021': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv', '2020': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv', '2019': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv', '2018': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv', '2017': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/300221

In [7]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")
tomorrow_datestring = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

print(today_datestring, thirty_days_ago_datestring, tomorrow_datestring)

2023-09-25 2023-08-26 2023-09-26


In [8]:
#set model folder constant
MODEL_FOLDER = './daily_models'

## Load extra features

In [9]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'


##Define several models

In [10]:
linear_tree_model = Boston311SurvDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type','queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':tomorrow_datestring},
                            files_dict=latest_URLS)

In [11]:
logistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [12]:
oldlogistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [13]:
logistic_tree_model = Boston311EventDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [14]:
kerasNLP_model = Boston311KerasNLP(train_date_range={'start':'2022-03-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue', 'source', 'subject', 'reason', 'department', 'ward_number'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':tomorrow_datestring},
                            files_dict=latest_URLS)

In [15]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [16]:
mydata = None

import pandas as pd
import numpy as np
import pickle

case_data_file = 'case_data.pkl'
mydata = None

X = None

if os.path.exists(case_data_file):
    mydata = pickle.load(open(case_data_file, "rb"))
else:
    mydata = kerasNLP_model.load_data()

    pickle.dump(mydata, open(case_data_file, "wb"))




In [17]:
mydata['case_enquiry_id']

43677     101004204966
43678     101004204967
43679     101004204970
43680     101004204968
43681     101004204972
              ...     
476951    101005013781
476952    101005013782
476953    101005013783
476954    101005013785
476955    101005013791
Name: case_enquiry_id, Length: 433279, dtype: int64

In [18]:
mydata = kerasNLP_model.enhance_data(mydata)


In [19]:
mydata = kerasNLP_model.apply_scenario(mydata)


In [20]:

mydata = kerasNLP_model.clean_data(mydata)

In [21]:
print(mydata['case_enquiry_id'])

43677     101004204966
43678     101004204967
43679     101004204970
43680     101004204968
43681     101004204972
              ...     
476951    101005013781
476952    101005013782
476953    101005013783
476954    101005013785
476955    101005013791
Name: case_enquiry_id, Length: 383627, dtype: int64


In [22]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X['cls_embedding'] = X['cls_embedding'].apply(literal_eval).apply(np.array)
    X['pooled_embedding'] = X['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [23]:
#load second file form pickle or from csv
pickle_file2022 = 'dataframe2.pkl'
EXTRA_mydata_FILE_2022 = './cls_and_pooled_embeddings_with_service_id_2022.csv'

X2022 = None

if os.path.exists(pickle_file2022):
    X2022 = pickle.load(open(pickle_file2022, "rb"))
else:
    X2022 = pd.read_csv(EXTRA_mydata_FILE_2022)

    #rename service_request_id to case_enquiry_id
    X2022.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X2022 = X2022[X2022['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X2022['case_enquiry_id'] = X2022['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X2022['cls_embedding'] = X2022['cls_embedding'].apply(literal_eval).apply(np.array)
    X2022['pooled_embedding'] = X2022['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X2022, open(pickle_file2022, "wb"))

In [24]:
#print information about X2022
print(X.info())
print(X2022.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275503 entries, 0 to 275502
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   case_enquiry_id   275503 non-null  object
 1   cls_embedding     275503 non-null  object
 2   pooled_embedding  275503 non-null  object
dtypes: object(3)
memory usage: 6.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305359 entries, 0 to 305358
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   case_enquiry_id   305359 non-null  int64 
 1   cls_embedding     305359 non-null  object
 2   pooled_embedding  305359 non-null  object
dtypes: int64(1), object(2)
memory usage: 7.0+ MB
None


In [25]:
#concatenate the two dataframes and reindex
df = pd.concat([X, X2022], ignore_index=True)

In [26]:
df.shape

(580862, 3)

In [27]:

# Assuming df is your DataFrame and it has columns 'cls_embedding' and 'pooled_embedding'
cls_embedding_flattened = np.stack(df['cls_embedding'].to_numpy())
pooled_embedding_flattened = np.stack(df['pooled_embedding'].to_numpy())

# Remove the old columns
df.drop(['cls_embedding', 'pooled_embedding'], axis=1, inplace=True)

# Add the new flattened columns
df_cls = pd.DataFrame(cls_embedding_flattened, columns=[f'cls_{i}' for i in range(cls_embedding_flattened.shape[1])])
df_pooled = pd.DataFrame(pooled_embedding_flattened, columns=[f'pooled_{i}' for i in range(pooled_embedding_flattened.shape[1])])

df = pd.concat([df, df_cls, df_pooled], axis=1)

In [28]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [29]:
df = df[is_numeric]

In [30]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [31]:
df.shape

(580861, 257)

In [32]:
mydata.shape

(383627, 438)

In [33]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [34]:
new_mydata.shape

(305945, 694)

In [35]:

df, y = kerasNLP_model.split_data(new_mydata)

In [36]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')

In [37]:
#list the number of rows in X and y
print(df.shape)
print(y.shape)


(305945, 691)
(305945,)


In [38]:
#best_model, best_hyperparameters = kerasNLP_model.tune_model(df, y, '/home/briarmoss/Documents/Boston_311/models/tuning')

In [39]:
#define hyperparameters
from kerastuner import HyperParameters

#set constants
start_nodes = 128
end_nodes = 64
#l2_0 = 0.00001
#learning_rate = 7.5842e-05
l2_0 = 0.001
learning_rate = 0.0001


hp = HyperParameters()
hp.Fixed('start_nodes', start_nodes)
hp.Fixed('end_nodes', end_nodes)
hp.Fixed('l2_0', l2_0)
hp.Fixed('learning_rate', learning_rate)

kerasNLP_model.best_hyperparameters = hp

In [40]:
#free all unused dataframes
df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, X2022, new_mydata, is_numeric, mydata]

for data_frame in df_to_delete:
    del data_frame

In [41]:

#parse CLS embedding column as array
test_acc = kerasNLP_model.train_model( df, y )

Starting Training at 2023-09-25 14:18:06.026850


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               88576     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 97417 (380.54 KB)
Trainable params: 97417 (380.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
<class 'pandas.core.frame.DataFrame'> (244756, 9)
<class 'pandas.core.frame.DataFrame'> (61189, 9)
run fit

Epoch 1/100


2023-09-25 14:18:09.349477: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1353011168 exceeds 10% of free system memory.


7634/7649 [============================>.] - ETA: 0s - loss: 1.3026 - accuracy: 0.6204 - top_k_categorical_accuracy: 0.7824

2023-09-25 14:18:20.494377: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 338252792 exceeds 10% of free system memory.


7649/7649 [==============================] - 12s 1ms/step - loss: 1.3025 - accuracy: 0.6204 - top_k_categorical_accuracy: 0.7824 - val_loss: 1.1941 - val_accuracy: 0.6559 - val_top_k_categorical_accuracy: 0.8118
Epoch 2/100
7649/7649 [==============================] - 11s 1ms/step - loss: 1.1182 - accuracy: 0.6737 - top_k_categorical_accuracy: 0.8265 - val_loss: 1.0858 - val_accuracy: 0.6878 - val_top_k_categorical_accuracy: 0.8317
Epoch 3/100
7649/7649 [==============================] - 11s 1ms/step - loss: 1.0531 - accuracy: 0.6938 - top_k_categorical_accuracy: 0.8379 - val_loss: 1.0245 - val_accuracy: 0.7059 - val_top_k_categorical_accuracy: 0.8425
Epoch 4/100
7649/7649 [==============================] - 11s 1ms/step - loss: 1.0093 - accuracy: 0.7088 - top_k_categorical_accuracy: 0.8461 - val_loss: 0.9950 - val_accuracy: 0.7123 - val_top_k_categorical_accuracy: 0.8480
Epoch 5/100
7649/7649 [==============================] - 11s 1ms/step - loss: 0.9755 - accuracy: 0.7205 - top_k_cate

2023-09-25 14:37:03.089459: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 338252792 exceeds 10% of free system memory.


1913/1913 [==============================] - 1s 584us/step - loss: 0.7421 - accuracy: 0.7792 - top_k_categorical_accuracy: 0.8813
Testing accuracy: 0.779192328453064 
Top-2 accuracy: 0.8812695145606995 
Test loss: 0.7420598268508911
Ending Training at 2023-09-25 14:37:04.327930
Training took 0:18:58.301080


## Train several models

In [42]:
print("learning is fun!") 

learning is fun!


In [43]:
#logistic_tree_model.run_pipeline()

In [44]:
#logistic_model.run_pipeline()

In [45]:
import gc
gc.collect()

2290

In [46]:
#linear_tree_model.run_pipeline()

In [47]:
import datetime

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)

# List of models
models = [kerasNLP_model]


# Iterate over models and save
for model in models:
    save_model_to_dir(model, model.model_type)


/home/briarmoss/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
"""
data = kerasNLP_model.load_data( 'predict' )
data = kerasNLP_model.enhance_data( data, 'predict')
clean_data = kerasNLP_model.clean_data_for_prediction( data )

X_predict, y_predict = kerasNLP_model.split_data( clean_data )
y_predict = kerasNLP_model.model.predict(X_predict)
data['survival_prediction'] = y_predict
return data
"""


"\ndata = kerasNLP_model.load_data( 'predict' )\ndata = kerasNLP_model.enhance_data( data, 'predict')\nclean_data = kerasNLP_model.clean_data_for_prediction( data )\n\nX_predict, y_predict = kerasNLP_model.split_data( clean_data )\ny_predict = kerasNLP_model.model.predict(X_predict)\ndata['survival_prediction'] = y_predict\nreturn data\n"

In [49]:

import datetime

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)

# List of models
models = [linear_tree_model, logistic_tree_model, logistic_model]


# Iterate over models and save
for model in models:
    save_model_to_dir(model, model.model_type)



AttributeError: 'NoneType' object has no attribute 'save'